# Adders generator

In [16]:
import math 
from datetime import datetime 
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

In [17]:
####################################################
# Inputs definitions
####################################################
INPUT_N = 4                                     # Input size is a INPUT_N x INPUT_N block of 4x4 sub-blocks
INPUT_COORD_WIDTH = 8                           # Number of bits as input for each block in an input line
INPUT_MACROBLOCK_SIZE = 16                      # Number of pixels in one macroblock (16 = 4x4)
PIXEL_WIDTH = 8                                 # Number of bits for each integer pixel from the reference frame


####################################################
# Interpolator definitions
####################################################
INPUT_LINE_SIZE = 9                             # Input size for ONE line loaded to the buffer (9 integer pixels)
TAPS = 6                                        # Number of taps
FILTERS = 15                                    # Number of filters (= number of outputs expected)
OUT_1ST_INTERP_WIDTH = 11                       # Width for every 15 interpolation output for FIRST PASS
OUT_2ND_INTERP_WIDTH = 14                       # Width for every 15 interpolation output for SECOND PASS


####################################################
# Motion Vector Generator definitions
####################################################
INPUT_COORD_WIDTH = 8                           # Input size for the coordinates X and Y
INPUT_MV_WIDTH = 8                              # Input size for the block motion vectors (MV0 and MV1)
OUTPUT_MV_GEN_WIDTH = 19                        # Output size of the Motion Vector module (4 and 6 parameters are equal)

In [18]:
####################################################
# Rewrite adders
####################################################
# [ADDER NAME, INPUT BIT WIDTH, DESCRIPTION, SIGNED/UNSIGNED]
ADDER_MATRIX = [['add_coords', INPUT_COORD_WIDTH, 'Adds the original coord value plus 1. Works for X and Y coords.', 'signed'],
              ['add_count_block', int(math.log2(INPUT_MACROBLOCK_SIZE)), 'Adder to control which sample will be processed.', 'unsigned'],
              ['add_counter', 4, 'Adder to control the loops. It must count until 13. And its result also selects the buffer column to be read.', 'unsigned']
              ]

In [19]:
###############################################
# Adder text template
###############################################
def generate_adder(ind):    
    # Modified date
    DATE_MODIF = str(datetime.today().strftime('%d/%m/%Y'))

    # Adder file text
    adder_verilog = '''/*-----------------------------------------------------------------------------------
* File: '''+ADDER_MATRIX[ind][0]+'''.v
* Date generated: 25/03/2023
* Date modified: '''+DATE_MODIF+'''
* Author: Bruna Suemi Nagai
* Description: '''+str(ADDER_MATRIX[ind][2])+'''
*----------------------------------------------------------------------------------- */

module '''+ADDER_MATRIX[ind][0]+''' (
  input '''+ADDER_MATRIX[ind][3]+''' ['''+str(ADDER_MATRIX[ind][1]-1)+''':0] A,
  input '''+ADDER_MATRIX[ind][3]+''' ['''+str(ADDER_MATRIX[ind][1]-1)+''':0] B,
  output reg '''+ADDER_MATRIX[ind][3]+''' ['''+str(ADDER_MATRIX[ind][1]-1)+''':0] C
);
    

// ------------------------------------------
// Combinational logic
// ------------------------------------------
always @ (A or B) begin
  C = A + B;
end
endmodule // '''+ADDER_MATRIX[ind][0]+'''
    '''

    # Write the adder file
    with open('../verilog/adders/'+ADDER_MATRIX[ind][0]+'.v', 'w') as adder_file:
        adder_file.write(adder_verilog)


In [20]:
# Define the list of words to select from
adder_names = ['Select an adder...', 'add_coords', 'add_count_block', 'add_counter', 'Generate all']

# Define the initial values 
SELECTED_ADDER = 'ADDER_NOOB'
ADDER_INDEX = 0

# Create the dropdown widget
word_select = widgets.Dropdown(
    options=adder_names,
    value=adder_names[0],
    description='Adder:')

# Create the button widget
button = widgets.Button(description='Generate adder file')

# Define a function to update the SELECTED_MUX variable
def update_adder_name(b):
    global ADDER_INDEX
    SELECTED_ADDER = word_select.value
    if SELECTED_ADDER == 'add_coords':
        ADDER_INDEX = 0
    elif SELECTED_ADDER == 'add_count_block':
        ADDER_INDEX = 1
    elif SELECTED_ADDER == 'add_counter':
        ADDER_INDEX = 2
    elif SELECTED_ADDER == 'Generate all':
        ADDER_INDEX = 3
    else:
        ADDER_INDEX = 4

    # Loop for generate all    
    if ADDER_INDEX == 3:
        for i in range(ADDER_INDEX):
            generate_adder(i)
            generate_adder(i)
            print('Register ', ADDER_MATRIX[i][0], ' generated.')
            print('Index =', i)
        print('Finished generating all adders :)')
    else:
        print('Adder', SELECTED_ADDER, 'generated.')
        print('Index =', ADDER_INDEX)
        generate_adder(ADDER_INDEX)

# Attach the function to the button's on-click event
button.on_click(update_adder_name)

# Display the widgets
display(word_select)
display(button)


Dropdown(description='Adder:', options=('Select an adder...', 'add_coords', 'add_count_block', 'add_counter', …

Button(description='Generate adder file', style=ButtonStyle())

Adder add_coords generated.
Index = 0
